In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
print(tf.__version__)
tf.disable_v2_behavior()

1.15.2
Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
import numpy as np
import _pickle
%matplotlib inline
import matplotlib.pyplot as plt
import math

In [0]:
import gym
env = gym.make('CartPole-v0')

In [0]:
# гиперпараметры
H = 10                 # кол-во нейронов в скрытом слое
batch_size = 5         
learning_rate = 1e-2   
gamma = 0.99           # дисконт для награды
D = 4                  # входная размерность

In [6]:
tf.reset_default_graph()

# дает вероятность выбора движения вправа или влева
observations = tf.placeholder(tf.float32, [None,D] , name="input_x")

W1 = tf.get_variable("W1", shape=[D, H],
           initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observations,W1))

W2 = tf.get_variable("W2", shape=[H, 1],
           initializer=tf.contrib.layers.xavier_initializer())
score = tf.matmul(layer1,W2)

probability = tf.nn.sigmoid(score)

# для обучения политики
tvars = tf.trainable_variables()                                 # весы
input_y = tf.placeholder(tf.float32,[None,1], name="input_y")    
advantages = tf.placeholder(tf.float32,name="reward_signal")

# loss функция послает весов в направлении движения,
# который дал хороший преимущество
loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages) 
newGrads = tf.gradients(loss,tvars)

adam = tf.train.AdamOptimizer(learning_rate=learning_rate) 
W1Grad = tf.placeholder(tf.float32,name="batch_grad1") 
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
# функция преимущество
def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

Загрузка агента и среды

In [9]:
xs,drs,ys = [],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.global_variables_initializer()

# Запускаем граф
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset() # инициализация наблюдения среды

    # Собираем градиенты в gradBuffer 
    # до тех пор будет готов обновить сеть политики
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # # просмотр
        # if reward_sum/batch_size > 100 or rendering == True : 
        #     env.render()
        #     rendering = True
            
        x = np.reshape(observation,[1,D])
        
        # загружем сеть политики и получаем движение
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x) # наблюдение
        y = 1 if action == 0 else 0 
        ys.append(y)

        # шаг среды и получаем новые измерения
        observation, reward, done, info = env.step(action)
        reward_sum += reward

        drs.append(reward) # записываем награду

        if done: 
            episode_number += 1
            # складываем все
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            xs,drs,ys = [],[],[] # очистка памяти

            # считаем награду со скидкой назад во времени
            discounted_epr = discount_rewards(epr)
            # нормализавать награду
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            # получаем градиент данного эпизода и сохраняем вgradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # если завершили достаточно эпизодов,
            # тогда обновляем сеть политики с нашими градиетами
            if episode_number % batch_size == 0: 
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
              
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print('Average reward for episode %.3f.  Total average reward %.3f' % (reward_sum/batch_size, running_reward/batch_size))
                
                if reward_sum/batch_size > 200: 
                    print("Task solved in",episode_number,'episodes')
                    break
                    
                reward_sum = 0
            
            observation = env.reset()
        
print(episode_number,'Episodes completed.')

Average reward for episode 9.000.  Total average reward 9.000
Average reward for episode 17.400.  Total average reward 9.084
Average reward for episode 13.200.  Total average reward 9.125
Average reward for episode 19.000.  Total average reward 9.224
Average reward for episode 17.800.  Total average reward 9.310
Average reward for episode 13.200.  Total average reward 9.349
Average reward for episode 17.400.  Total average reward 9.429
Average reward for episode 22.400.  Total average reward 9.559
Average reward for episode 12.800.  Total average reward 9.591
Average reward for episode 20.400.  Total average reward 9.699
Average reward for episode 17.800.  Total average reward 9.780
Average reward for episode 21.000.  Total average reward 9.892
Average reward for episode 22.800.  Total average reward 10.022
Average reward for episode 19.600.  Total average reward 10.117
Average reward for episode 12.800.  Total average reward 10.144
Average reward for episode 15.400.  Total average rew